# Clean Lat/Lon
### Handle locations that could not be geocoded
* See https://github.com/geopandas/geopandas/pull/1610/
* Manually change the local Python file for `geopandas.tools.geocoding.py` as the merged change is not currently released

In [31]:
import geopandas as gpd
import numpy as np
import pandas as pd

## Read survey data

In [8]:
data = gpd.read_file('raw/survey.csv')
print('Total responses: ', data.shape[0])

Total responses:  3034


## Clean up

In [17]:
# Replace blanks with NaN
data.Location = data.Location.str.strip().str.upper().replace('', np.nan)

# Copy non-null locations into a new series
locations = data.Location.dropna()

print('Responses with non-empty locations:', locations.shape[0])

Responses with non-empty locations: 2783


## Geocode with Nominatim
This takes around 20 minutes

In [ ]:
geocoded_dfs = []

for idx in range(0, locations.shape[0], 50):
    print(idx)
    
    df = gpd.tools.geocode(
        locations.iloc[ idx : idx+50 ],
        provider='nominatim',
        user_agent='testing_nominatim'
    )
    geocoded_dfs.append( df )

## Join all geocoded dataframes together

In [102]:
geocoded = pd.concat(geocoded_dfs)

print( "Successfully geocoded: ", geocoded[~geocoded.geometry.is_empty].geometry.count() )

Successfully geocoded:  2735


## Assign continents and save

In [103]:
# Read continents shapefile
continents = gpd.read_file('raw/continents.geojson')

# Perform spatial join
final = gpd.sjoin(geocoded, continents, how='left')

# Create Lat & Lng columns
final['Lat'] = final.geometry.apply(lambda x: x.coords[0][1] if x else None)
final['Lng'] = final.geometry.apply(lambda x: x.coords[0][0] if x else None)


# Save
final.to_csv('data/data.csv', index=False)

## Counts by continent

In [75]:
final.CONTINENT.value_counts()

Asia             904
North America    782
Europe           510
South America    168
Africa           147
Australia         70
Oceania           11
Name: CONTINENT, dtype: int64

# Prepare scatterplot data

In [108]:
scatter = pd.read_csv(
    'raw/survey.csv',
    usecols=['Years of school', 'Experience with data visualization']
).dropna()

In [112]:
scatter.groupby(
    ['Experience with data visualization', 'Years of school']
).size().reset_index().to_csv('data/scatter.csv', index=False)

In [115]:
scatter['Experience with data visualization'].value_counts()

1.0    1097
2.0     686
3.0     676
4.0     215
5.0      79
Name: Experience with data visualization, dtype: int64

In [116]:
scatter['Years of school'].value_counts()

20.0    1333
16.0     775
18.0     176
17.0     116
15.0      93
12.0      73
19.0      57
14.0      49
13.0      25
10.0      22
6.0       13
0.0        7
8.0        5
11.0       4
7.0        3
9.0        2
Name: Years of school, dtype: int64

In [126]:
scatter[ scatter['Years of school'] >= 16 ].count()

Experience with data visualization    2457
Years of school                       2457
dtype: int64

In [124]:
scatter.count()

Experience with data visualization    2753
Years of school                       2753
dtype: int64

In [127]:
2457/2753

0.8924809298946603